# Group Work Project - M7

June 27, 2021

- **Course**: MScFE 620 Discrete-time Stochastic Processes
- **Professors**:
    - Sergio Garcia
    - Ivan Blanko
- **Class**: C21-S1
- Group 2, with members:
    - Juan Antonio Vargas Lopez; juanvarl@hotmail.com
    - Kanittha Setthapitayakul; kanittha.se@hotmail.com
    - Loc Nguyen; vinhloc30796@gmail.com

### 1. First, consider the case of an American Call option with underlying stock trading at \\$95\, Strike price of \\$90 and 1 year maturity. Use the Binomial model with the parameters r=0 and 5 steps in the pricing process. Additionally, set the upper movement of the price (u) to an expression that will depend on the number on your group:

<center>$u=(1.1+\frac{Group Number}{100})$

And define  <center> $d=\frac{1}{u}$

Consider American option

In [1]:
import numpy as np
np.set_printoptions(precision=4, suppress=True) # prettify numpy
import pandas as pd
pd.set_option('precision', 4) # prettify pandas

Define initial input parameters as below:

In [2]:
# Set starting price at $95
s0 = 95
# Set strike price at $90
k = 90
r = 0
n = 5

In [3]:
# Calculate u, d, P_star
group_number = 2
u = 1.10 + group_number/100; d = 1/u
pu = (1 - d) / (u - d)
pd = 1-pu

print('pu='+str(pu))
print('pd='+str(pd))

pu=0.4716981132075471
pd=0.5283018867924529


Create Binomial tree regarding to initial parameters:

In [4]:
def stocktree(n,s0,u,d): 
    n_stocktree = np.zeros((n+1,n+1))
    for j in range(n+1):
        for i in range(j+1):
            n_stocktree[i][j] = s0*(u**i)*(d**(j-i))
    return n_stocktree

In [5]:
step5_stocktree = stocktree(n,s0,u,d)
step5_stocktree

array([[ 95.    ,  84.8214,  75.7334,  67.6191,  60.3742,  53.9056],
       [  0.    , 106.4   ,  95.    ,  84.8214,  75.7334,  67.6191],
       [  0.    ,   0.    , 119.168 , 106.4   ,  95.    ,  84.8214],
       [  0.    ,   0.    ,   0.    , 133.4682, 119.168 , 106.4   ],
       [  0.    ,   0.    ,   0.    ,   0.    , 149.4843, 133.4682],
       [  0.    ,   0.    ,   0.    ,   0.    ,   0.    , 167.4225]])

Create function to calculate European option price in order to compare price between American and European option price

In [6]:
def european_payofftree(bi_tree,n,k,iscall):
    payoff_tree = np.zeros((n+1,n+1))
    iopt = 0
    if iscall:
        iopt = 1
    else:
        iopt = -1
    for i in range(n+1):
        payoff_tree[i][n] = np.maximum(0, iopt * (bi_tree[i][n]-K))
    return payoff_tree

def probtree(n,pu,pd):
    import math as m
    prob_tree = np.zeros((n+1,n+1))
    for j in range(n+1):
        for i in range(j+1):
            prob_tree[i][j] = m.factorial(j) / (m.factorial(i) * m.factorial(j-i))* pu**i * pd**(j-i)
    return prob_tree
def european_price(eu_payofftree,prob_tree,n):
    finalpayoff = 0.0
    for i in range(n+1):
        finalpayoff = finalpayoff + eu_payofftree[i][n] * prob_tree[i][n]
        optionprice = finalpayoff
    return finalpayoff

Calculate option payoff at each nodes

<center>$g_{Call}=max(S_0-K,0)$

<center>$g_{Put}=max(K-S_0,0)$

In [7]:
def american_payofftree(bi_tree,n,k,iscall):
    payoff_tree = np.zeros((n+1,n+1))
    # Get payoffs at each node
    if iscall:
        iopt = 1
    else:
        iopt = -1
    for j in range(n+1):
        for i in range(j+1):
            payoff_tree[i][j] = np.maximum(0,iopt*(bi_tree[i][j]-k))
    return payoff_tree

Calculate value of option at each nodes from backward recursion of option payoff:

<center>$v_{k} = max(\frac{1}{1+r}[g_{u}(p_{u})+g_{d}(p_{d})],g_{k})$

In [8]:
def american_optprctree(payoff_tree,pu,pd,K,iscall,r):
    opt_prc_tree = payoff_tree
    if iscall:
        iopt = 1
    else:
        iopt = -1
    for j in range(n-1,-1,-1): 
        for i in range(j+1):
            opt_prc_tree[i][j] = (1/(1+r))*(pu * payoff_tree[i+1][j+1]+ pd * payoff_tree[i][j+1])
            opt_prc_tree[i][j] = np.maximum(payoff_tree[i][j],opt_prc_tree[i][j])
    return opt_prc_tree     

In [9]:
def e_americanoptprc(prob_tree,us_pricetree,n):
    for j in range(n+1):
        expect_price = 0
        for i in range(j+1):
            expect_price = expect_price + us_pricetree[i][j] * prob_tree[i][j]
        print('step '+str(j)+': '+str(expect_price))

#### a. In this setting, find the value of the derivative at each node.

#### ANSWER

In [10]:
iscall = True
call_us_payofftree = american_payofftree(step5_stocktree,n,k,iscall)
call_us_payofftree

array([[ 5.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    , 16.4   ,  5.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    , 29.168 , 16.4   ,  5.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    , 43.4682, 29.168 , 16.4   ],
       [ 0.    ,  0.    ,  0.    ,  0.    , 59.4843, 43.4682],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    , 77.4225]])

In [11]:
call_us_optprctree = american_optprctree(call_us_payofftree,pu,pd,k,iscall,r)
call_us_optprctree

array([[12.2963,  5.7892,  1.7212,  0.    ,  0.    ,  0.    ],
       [ 0.    , 19.5842, 10.3454,  3.649 ,  0.    ,  0.    ],
       [ 0.    ,  0.    , 29.9316, 17.8454,  7.7358,  0.    ],
       [ 0.    ,  0.    ,  0.    , 43.4682, 29.168 , 16.4   ],
       [ 0.    ,  0.    ,  0.    ,  0.    , 59.4843, 43.4682],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    , 77.4225]])

In [12]:
print('American call option price is '+str(call_us_optprctree[0][0]))

American call option price is 12.296267645333053


#### b. Is there any point time where we, as buyers of the option, benefit from early exercise?

#### Answer: 

From above, we can see that there is a possibility that the option price is more than different between Spot price and Strike price. Thus, in this case, there is benefit from early exercise.

#### c. Explain whether your previous answer is (or is not) always the case.

#### ANSWER:

For an american call option, early excercise is never benefit to buyer in case the interest rate (r) and the time value applied . For example, if r = 10%, the option price will be

In [13]:
call_us_optprctree = american_optprctree(call_us_payofftree,pu,pd,k,iscall,0.1)
call_us_optprctree

array([[ 7.635 ,  3.9541,  1.2932,  0.    ,  0.    ,  0.    ],
       [ 0.    , 13.3762,  7.7726,  3.0157,  0.    ,  0.    ],
       [ 0.    ,  0.    , 22.488 , 14.7482,  7.0326,  0.    ],
       [ 0.    ,  0.    ,  0.    , 35.9241, 26.5164, 16.4   ],
       [ 0.    ,  0.    ,  0.    ,  0.    , 54.0767, 43.4682],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    , 77.4225]])

We can see that when the spot price more than strike price the option value is lower than its payoff. However, the american call option might be benefit to buyer especially with dividend paying stock.

### 2. Consider the same parameters and setting as in (1), but now for pricing an American Put Option.

#### a. Find the value of the derivative at each node.

#### ANSWER:

In [14]:
iscall = False
put_us_payofftree = american_payofftree(step5_stocktree,n,k,iscall)
put_us_payofftree

array([[ 0.    ,  5.1786, 14.2666, 22.3809, 29.6258, 36.0944],
       [ 0.    ,  0.    ,  0.    ,  5.1786, 14.2666, 22.3809],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  5.1786],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ]])

In [15]:
put_us_optprctree = american_optprctree(put_us_payofftree,pu,pd,k,iscall,r)
put_us_optprctree

array([[ 7.2963, 10.9678, 15.9878, 22.3809, 29.6258, 36.0944],
       [ 0.    ,  3.1842,  5.3454,  8.8276, 14.2666, 22.3809],
       [ 0.    ,  0.    ,  0.7636,  1.4454,  2.7358,  5.1786],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ]])

In [16]:
print('American put option price is '+str(put_us_optprctree[0][0]))

American put option price is 7.2962676453330495


#### b. Is there any point time where we, as buyers of the option, benefit from early exercise?

#### ANSWER:

From above, we can see that there is a possibility that the option price is more than different between Spot price and Strike price. Thus, in this case, there is benefit from early exercise.

#### c. Explain whether your previous answer is (or is not) always the case.

#### ANSWER:

The American Put option will not be benefit to buyer to early exercise if the spot price is higher than the exercise boundary. 